In [ ]:
import multiprocessing
from airo_camera_toolkit.cameras.multiprocess.multiprocess_stereo_rgbd_camera import (
    MultiprocessStereoRGBDPublisher,
    MultiprocessStereoRGBDReceiver,
)
from airo_camera_toolkit.cameras.zed.zed2i import Zed2i

create_multiprocess_camera = False

# Setting up the camera
camera_kwargs = {
    "resolution": Zed2i.RESOLUTION_2K,
    "depth_mode": Zed2i.NEURAL_DEPTH_MODE,
    "fps": 15,
}
camera_publisher = None

if create_multiprocess_camera:
    multiprocessing.set_start_method("spawn")

    # Running the camera in a seperate process enables us to record videos even if the main process is blocking
    camera_publisher = MultiprocessStereoRGBDPublisher(Zed2i, camera_kwargs)
    # self.camera_publisher.publish_depth_image = False

    camera_publisher.start()
    camera = MultiprocessStereoRGBDReceiver("camera")
else:
    camera = Zed2i(**camera_kwargs)

In [ ]:
import time
from loguru import logger

# camera.runtime_params.disable_depth = True

camera_period = 1 / camera.fps
timestamp_prev_grab = time.time()

for _ in range(150):
    camera._grab_images()
    timestamp = time.time()
    if timestamp_prev_grab is not None:
        publish_period = timestamp - timestamp_prev_grab

        if publish_period > 1.1 * camera_period:
            logger.warning(
                f"Time since previous grab: {publish_period:.3f} s. Camera cannot maintain {camera.fps:.2f} fps. "
            )
        else:
            logger.debug(f"Time since previous grab: {publish_period:.3f} s")
    timestamp_prev_grab = timestamp

In [ ]:
%%timeit
camera.get_rgb_image_as_int()

In [ ]:
camera.get_rgb_image_as_int()

In [ ]:
%%timeit
camera._retrieve_colored_point_cloud()

In [ ]:
import pyzed.sl as sl

point_cloud_matrix = sl.Mat()

In [ ]:
%%timeit
camera.camera.retrieve_measure(point_cloud_matrix, sl.MEASURE.XYZ)

In [ ]:
%%timeit
camera.point_cloud_matrix.get_data()

In [ ]:
%%timeit
depth_map = camera._retrieve_depth_map()
depth_image = camera._retrieve_depth_image()
confidence_map = camera._retrieve_confidence_map()
point_cloud = camera._retrieve_colored_point_cloud()

In [ ]:
image_matrix = sl.Mat()
image_matrix_right = sl.Mat()
depth_image_matrix = sl.Mat()
depth_matrix = sl.Mat()
point_cloud_matrix = sl.Mat()
confidence_matrix = sl.Mat()

In [ ]:
%%timeit
# camera.camera.retrieve_image(image_matrix, sl.VIEW.LEFT)
# camera.camera.retrieve_image(image_matrix_right, sl.VIEW.RIGHT)
camera.camera.retrieve_image(depth_image_matrix, sl.VIEW.DEPTH)
camera.camera.retrieve_measure(depth_matrix, sl.MEASURE.DEPTH)
camera.camera.retrieve_measure(confidence_matrix, sl.MEASURE.CONFIDENCE)
camera.camera.retrieve_measure(point_cloud_matrix, sl.MEASURE.XYZ)

In [ ]:
depth_map = camera._retrieve_depth_map()
depth_image = camera._retrieve_depth_image()
confidence_map = camera._retrieve_confidence_map()
point_cloud = camera._retrieve_colored_point_cloud()

In [ ]:
import threading
import numpy as np

depth_map_array = np.zeros_like(depth_map)


def retrieve_depth_map_to_array():
    global depth_map_array
    depth_map_array = camera._retrieve_depth_map()


retrieve_depth_map_thread = threading.Thread(target=retrieve_depth_map_to_array)
retrieve_depth_map_thread.start()
retrieve_depth_map_thread.join()

import matplotlib.pyplot as plt

plt.imshow(depth_map_array)

In [ ]:
%%timeit
camera._retrieve_depth_map()

In [ ]:
%%timeit
retrieve_depth_map_thread = threading.Thread(target=retrieve_depth_map_to_array)
retrieve_depth_map_thread.start()
retrieve_depth_map_thread.join()

In [ ]:
depth_map_array = np.zeros_like(depth_map)
depth_image_array = np.zeros_like(depth_image)
confidence_map_array = np.zeros_like(confidence_map)
point_cloud_positions_array = np.zeros_like(point_cloud.points)
point_cloud_color_array = np.zeros_like(point_cloud.colors)


def retrieve_depth_map_to_array():
    global depth_map_array
    depth_map_array = camera._retrieve_depth_map()


def retrieve_depth_image_to_array():
    global depth_image_array
    depth_image_array = camera._retrieve_depth_image()


def retrieve_confidence_map_to_array():
    global confidence_map_array
    confidence_map_array = camera._retrieve_confidence_map()


def retrieve_point_cloud_to_array():
    global point_cloud_positions_array, point_cloud_color_array
    point_cloud = camera._retrieve_colored_point_cloud()
    point_cloud_positions_array = point_cloud.points
    point_cloud_color_array = point_cloud.colors

In [ ]:
%%timeit
retrieve_depth_map_thread = threading.Thread(target=retrieve_depth_map_to_array)
retrieve_depth_image_thread = threading.Thread(target=retrieve_depth_image_to_array)
retrieve_confidence_map_thread = threading.Thread(target=retrieve_confidence_map_to_array)
retrieve_point_cloud_thread = threading.Thread(target=retrieve_point_cloud_to_array)

retrieve_point_cloud_thread.start()
retrieve_depth_map_thread.start()
retrieve_depth_image_thread.start()
retrieve_confidence_map_thread.start()

retrieve_point_cloud_thread.join()
retrieve_depth_map_thread.join()
retrieve_depth_image_thread.join()
retrieve_confidence_map_thread.join()

In [ ]:
import time
from airo_camera_toolkit.interfaces import StereoRGBDCamera

times_retrieve = []
times_copy = []

period = 0.66

for i in range(20):
    time_start = time.time()

    camera._grab_images()
    rgb_left = camera._retrieve_rgb_image_as_int()
    rgb_right = camera._retrieve_rgb_image_as_int(StereoRGBDCamera.RIGHT_RGB)
    depth_map = camera._retrieve_depth_map()
    depth_image = camera._retrieve_depth_image()
    confidence_map = camera._retrieve_confidence_map()
    point_cloud = camera._retrieve_colored_point_cloud()

    time_retrieved = time.time()
    times_retrieve.append(time_retrieved - time_start)

    rgb_left.copy()
    rgb_right.copy()
    depth_map.copy()
    depth_image.copy()
    confidence_map.copy()
    point_cloud.points.copy()
    point_cloud.colors.copy()

    time_copied = time.time()
    times_copy.append(time_copied - time_retrieved)

    # Wait a bit
    while time.time() - time_start < period:
        time.sleep(0.0001)

times = np.array(times_retrieve) + np.array(times_copy)
print(f"Average time elapsed: {np.mean(times):.3f} s")
print(f"Average time elapsed (retrieve): {np.mean(times_retrieve):.3f} s")
print(f"Average time elapsed (copy): {np.mean(times_copy):.3f} s")

In [ ]:
rgb_left = camera._retrieve_rgb_image_as_int()
rgb_right = camera._retrieve_rgb_image_as_int(StereoRGBDCamera.RIGHT_RGB)
depth_map = camera._retrieve_depth_map()
depth_image = camera._retrieve_depth_image()
confidence_map = camera._retrieve_confidence_map()
point_cloud = camera._retrieve_colored_point_cloud()

In [ ]:
np.info(rgb_left)

In [ ]:
np.info(rgb_right)

In [ ]:
np.info(depth_map)

In [ ]:
# print the properties of the depth_image array
np.info(depth_image)

In [ ]:
np.info(confidence_map)

In [ ]:
np.info(point_cloud.points)

In [ ]:
np.info(point_cloud.colors)

In [ ]:
%%timeit
point_cloud.points.copy()

In [ ]:
point_cloud.points[0]

In [ ]:
camera.camera.retrieve_measure(point_cloud_matrix, sl.MEASURE.XYZ)
point_cloud_positions_XYZ_ = point_cloud_matrix.get_data()

In [ ]:
point_cloud_positions_XYZ_.shape

In [ ]:
point_cloud_positions_XYZ_[0]

In [ ]:
import cv2

cv2.cvtColor(point_cloud_positions_XYZ_, cv2.COLOR_BGRA2BGR)[0]

In [ ]:
%%timeit
cv2.cvtColor(point_cloud_positions_XYZ_, cv2.COLOR_BGRA2BGR)

In [ ]:
pcd_cv2 = cv2.cvtColor(point_cloud_positions_XYZ_, cv2.COLOR_BGRA2BGR)

In [ ]:
%%timeit
pcd_cv2.copy()

In [ ]:
camera.camera.retrieve_measure(point_cloud_matrix, sl.MEASURE.XYZ)

In [ ]:
%%timeit
np.ascontiguousarray(point_cloud_matrix.get_data()[:, :, :3])

In [ ]:
%%timeit
cv2.cvtColor(rgb_left, cv2.COLOR_RGB2BGR)

In [ ]:
from airo_camera_toolkit.utils.image_converter import ImageConverter

In [ ]:
%%timeit
ImageConverter.from_numpy_int_format(rgb_left).image_in_opencv_format